# callcenter

The Simplexville Emergency Dispatch receives urgent calls for the police and fire departments in Simplexville, as well as for neighboring towns Bellman and Fulkerson. 65% of the calls are from Simplexville, 20% of the calls are from Bellman, and 15% of the calls are from Fulkerson. The dispatch prioritizes calls by the location of the call: Simplexville calls get the highest priority, then Bellman, and then Fulkerson. Calls arrive according to an exponential interarrival distribution with a mean of 3.2 minutes. The time that a call requires is also exponentially distributed, with a mean of 11.3 minutes. There are always 4 operators on duty at the dispatch.

* Simulate this system for 24 hours. Using 100 replications, give point and interval estimates of:
    - the average delay for a call from Simplexville
    - the average delay for a call from Bellman
    - the average delay for a call from Fulkerson

In [1]:
##### Setup #####
# Import various functions from NumPy
from numpy import inf, mean, std, sqrt

# Import seed initializer and random sampling functions from NumPy
from numpy.random import seed, rand, exponential

# Import t random variable from Scipy
from scipy.stats import t

# Import bisect_right from bisect
from bisect import bisect_left

# Import everything from SimPy
from SimPy.Simulation import *

In [2]:
##### Parameters #####
class P:
    # Calls arrive according to exponential interarrival 
    # time distribution with mean 3.2 minutes
    interarrivalTimeMean = 3.2
    
    # Time required for calls is exponentially distributed 
    # with mean 11.3 minutes
    serviceTimeMean = 11.3
    
    # Number of operators
    nServers = 4
    
    # Simulate dispatch for 24 hours
    simulationTimeMax = 24 * 60
    
    
##### Distributions #####
class D:
    # Call location
    # 0 = Simplexville, 1 = Bellman, 2 = Fulkerson
    def callLocation():
        # List of possible values, including -inf
        a = [-inf, 0, 1, 2]

        # cdf at values
        cdf = [0, 0.65, 0.85, 1.0]

        # Generate variate
        variate = a[bisect_left(cdf, rand())]

        # Return variate
        return variate
    
    
##### Processes #####
# Call
class Call(Process):
    def behavior(self):
        # Determine call location
        callLocation = D.callLocation()
        
        # If the call is from Simplexville, give highest priority (100)
        if callLocation == 0:
            # "Start stopwatch" for delay monitor
            start = now()
            
            # Call arrives, joins queue with priority 100
            yield request, self, R.server, 100
            
            # "Stop stopwatch" for delay monitor
            finish = now()
            
            # Record delay
            M.delayS.observe(finish - start)

            # Call is released from queue and starts service
            serviceTime = exponential(scale = P.serviceTimeMean)
            yield hold, self, serviceTime

            # Call finishes service, leaves
            yield release, self, R.server
        
        # If the call is from Bellman, give middle priority (50)
        elif callLocation == 1:
            # "Start stopwatch" for delay monitor
            start = now()
            
            # Call arrives, joins queue with priority 50
            yield request, self, R.server, 50
            
            # "Stop stopwatch" for delay monitor
            finish = now()
            
            # Record delay
            M.delayB.observe(finish - start)           

            # Call is released from queue and starts service
            serviceTime = exponential(scale = P.serviceTimeMean)
            yield hold, self, serviceTime

            # Call finishes service, leaves
            yield release, self, R.server
            
        # If the call is from Fulkerson, give lowest priority (0)
        elif callLocation == 2:
            # "Start stopwatch" for delay monitor
            start = now()
            
            # Call arrives, joins queue with priority 0
            yield request, self, R.server, 0
            
            # "Stop stopwatch" for delay monitor
            finish = now()
            
            # Record delay
            M.delayF.observe(finish - start)            

            # Call is released from queue and starts service
            serviceTime = exponential(scale = P.serviceTimeMean)
            yield hold, self, serviceTime

            # Call finishes service, leaves
            yield release, self, R.server         

# Call generator
class CallGenerator(Process):
    def behavior(self):
        # At the start of the simulation, no calls have arrived
        nCalls = 0
        
        # Customer arrivals
        while True:
            # Wait until the next arrival
            interarrivalTime = exponential(scale = P.interarrivalTimeMean)
            yield hold, self, interarrivalTime
            
            # Create a new call
            c = Call(name="Call {0}".format(nCalls))
            
            # Activate the call's behavior
            activate(c, c.behavior())

            # Count this new call
            nCalls += 1

            
##### Resources #####
class R:
    # Server
    server = None
    

##### Monitors #####
class M:
    delayS = None
    delayB = None
    delayF = None


##### Simulation #####
def model(inputSeed):
    # Initialize SimPy 
    initialize()

    # Initialize seed for random number generator
    seed(inputSeed)

    # Create the server resource
    R.server = Resource(capacity = P.nServers, qType = PriorityQ)
    
    # Create delay monitors
    M.delayS = Monitor()
    M.delayB = Monitor()
    M.delayF = Monitor()

    # Activate the call generator
    e = CallGenerator()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)
    
    # Compute performance measures
    avgDelayS = M.delayS.mean()
    avgDelayB = M.delayB.mean()
    avgDelayF = M.delayF.mean()
    
    # Return performance measures
    return [avgDelayS, avgDelayB, avgDelayF]

In [3]:
##### Experiment #####
n = 100
avgDelaySObs = []
avgDelayBObs = []
avgDelayFObs = []
for i in range(n):
    [avgDelayS, avgDelayB, avgDelayF] = model(inputSeed = 123*i)
    avgDelaySObs.append(avgDelayS)
    avgDelayBObs.append(avgDelayB)
    avgDelayFObs.append(avgDelayF)

In [4]:
##### Analysis: average delay for calls from Simplexville #####
# Observed sample mean
avgDelaySSM = mean(avgDelaySObs)

# Observed sample standard deviation
avgDelaySSSD = std(avgDelaySObs, ddof = 1)

# Confidence level 0.05
alpha = 0.05
avgDelaySCIL = avgDelaySSM - t.ppf(1 - alpha/2, n - 1) * avgDelaySSSD / sqrt(n)
avgDelaySCIR = avgDelaySSM + t.ppf(1 - alpha/2, n - 1) * avgDelaySSSD / sqrt(n)

print("Average delay for calls from Simplexville:")
print("  Sample mean: {0}".format(avgDelaySSM))
print("  Sample standard deviation: {0}".format(avgDelaySSSD))
print("  {0}% confidence interval: [{1}, {2}]".format((1 - alpha)*100, avgDelaySCIL, avgDelaySCIR))

Average delay for calls from Simplexville:
  Sample mean: 4.811633164162375
  Sample standard deviation: 1.6754810766257144
  95.0% confidence interval: [4.479181368745099, 5.144084959579651]


In [5]:
##### Analysis: average delay for calls from Bellman #####
# Observed sample mean
avgDelayBSM = mean(avgDelayBObs)

# Observed sample standard deviation
avgDelayBSSD = std(avgDelayBObs, ddof = 1)

# Confidence level 0.05
alpha = 0.05
avgDelayBCIL = avgDelayBSM - t.ppf(1 - alpha/2, n - 1) * avgDelayBSSD / sqrt(n)
avgDelayBCIR = avgDelayBSM + t.ppf(1 - alpha/2, n - 1) * avgDelayBSSD / sqrt(n)

print("Average delay for calls from Bellman:")
print("  Sample mean: {0}".format(avgDelayBSM))
print("  Sample standard deviation: {0}".format(avgDelayBSSD))
print("  {0}% confidence interval: [{1}, {2}]".format((1 - alpha)*100, avgDelayBCIL, avgDelayBCIR))

Average delay for calls from Bellman:
  Sample mean: 18.623521997763138
  Sample standard deviation: 12.851727319736451
  95.0% confidence interval: [16.073460477364307, 21.17358351816197]


In [6]:
##### Analysis: average delay for calls from Fulkerson #####
# Observed sample mean
avgDelayFSM = mean(avgDelayFObs)

# Observed sample standard deviation
avgDelayFSSD = std(avgDelayFObs, ddof = 1)

# Confidence level 0.05
alpha = 0.05
avgDelayFCIL = avgDelayFSM - t.ppf(1 - alpha/2, n - 1) * avgDelayFSSD / sqrt(n)
avgDelayFCIR = avgDelayFSM + t.ppf(1 - alpha/2, n - 1) * avgDelayFSSD / sqrt(n)

print("Average delay for calls from Fulkerson:")
print("  Sample mean: {0}".format(avgDelayFSM))
print("  Sample standard deviation: {0}".format(avgDelayFSSD))
print("  {0}% confidence interval: [{1}, {2}]".format((1 - alpha)*100, avgDelayFCIL, avgDelayFCIR))

Average delay for calls from Fulkerson:
  Sample mean: 56.1898922788944
  Sample standard deviation: 52.56318794556716
  95.0% confidence interval: [45.76021542420128, 66.61956913358752]
